In [1]:
# Cell 1: Imports (run this first)

import os
import time
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
DATASET_DIR = "D:\HandSign\asl_alphabet_train"  
IMG_HEIGHT = 200
IMG_WIDTH = 200
BATCH_SIZE = 32
EPOCHS = 15

CLASSES = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["space", "del", "nothing"]

print("Classes:", CLASSES)
print("Num classes:", len(CLASSES))
